In [3]:
import pandas as pd
import os
import json
import numpy as np
# import streamlit as st
# import streamlit.components.v1 as components
import plotly.express as px
import plotly.graph_objs as go
import sys
DIR = os.getenv('KEA_BASE_DIR')
sys.path.append(DIR)
import application
from application import *

In [60]:
# Reading the GeoJSON
with open("AC_Boundary_simplified.json",'r',encoding='utf-8') as file:
    acmap = json.loads(file.read())

In [61]:
# This part of the JSON contains critical information
acmap['features'][0]['properties']

{'OBJECTID': 610,
 'AC_CODE': '1',
 'AC_NAME': 'Nippani',
 'PC_CODE': '01',
 'KGISDistri': '01',
 'AC_NAME_KN': 'ನಿಪ್ಪಾಣಿ',
 'Shape_STAr': 425991971.009,
 'Shape_STLe': 207237.810237}

In [62]:
# Fetch the constituency number and the corresponding constituency name from GeoJSON and convert to Dataframe

temp_dict = {}

for feature in acmap['features']:
    temp_dict.update({int(feature['properties']['AC_CODE']):feature['properties']['AC_NAME'].strip()})

In [63]:
# Fetch the constituency number and the corresponding constituency name from OpenCity dataset and convert to Dataframe

with open(f"{DIR}/2013 Elections\const_num.json",'r',encoding='utf-8') as file:
    const_num = json.loads(file.read())

In [66]:
# Correcting constituency names and numbers in the GeoJSON
for feature in acmap['features']:
    feature['properties']['AC_NAME'] = const_num['Constituency Name'][feature['properties']['AC_CODE'].lstrip('0')]
    feature['properties']['AC_CODE'] = feature['properties']['AC_CODE'].lstrip('0')

In [67]:
# Saving the JSON
with open("AC_Boundary_simplified.json",'w',encoding='utf-8') as file:
    file.write(json.dumps(acmap))

In [68]:
with open("BBMP.json",'r',encoding='utf-8') as file:
    bbmp = json.loads(file.read())

In [69]:
# Correcting constituency names and numbers in the GeoJSON
for feature in bbmp['features']:
    feature['properties']['AC_NAME'] = const_num['Constituency Name'][feature['properties']['AC_CODE'].lstrip('0')]
    feature['properties']['AC_CODE'] = feature['properties']['AC_CODE'].lstrip('0')

In [70]:
# Saving the JSON
with open("BBMP.json",'w',encoding='utf-8') as file:
    json.dump(bbmp, file)

In [71]:
# Define a scaling factor
scale_factor = 6.25

VERTICAL_OFFSET = 1.9
HORIZONTAL_OFFSET = 0.6

# Define center point of your map for scaling
# Update the code here to handle list of coordinates
num_coords = 0
total_lat = 0
total_lon = 0

for feature in bbmp['features']:
    for ring in feature['geometry']['coordinates']:
        for coord in ring:
            total_lat += coord[1]
            total_lon += coord[0]
            num_coords += 1

center_lat = total_lat / num_coords
center_lon = total_lon / num_coords

# Iterate over all coordinates and scale
for feature in bbmp['features']:
    for ring in feature['geometry']['coordinates']:
        for coord in ring:
            # Calculate offset from center point
            lat_offset = coord[1] - center_lat
            lon_offset = coord[0] - center_lon

            # Scale offset and recenter
            coord[1] = center_lat + (lat_offset * scale_factor) + VERTICAL_OFFSET
            coord[0] = center_lon + (lon_offset * scale_factor) + HORIZONTAL_OFFSET

# Save your scaled GeoJSON
with open('scaled_BBMP.json', 'w') as f:
    json.dump(bbmp, f)

In [72]:
with open('AC_Boundary_simplified.json', 'r') as f:
    acmap = json.load(f)

acmap['features'].extend(bbmp['features'])

# Save combined GeoJSON
with open('combined.json', 'w') as f:
    json.dump(acmap, f)